In [1]:
from __future__ import print_function
from functools import reduce
import os, sys, random, itertools, codecs
import json
import re
import tarfile
import tempfile

import numpy as np
np.random.seed(1337)  # for reproducibility

'''
300D Model - Train / Test (epochs)
=-=-=
Batch size = 512
Fixed GloVe
- 300D SumRNN + Translate + 3 MLP (1.2 million parameters) - 0.8315 / 0.8235 / 0.8249 (22 epochs)
- 300D GRU + Translate + 3 MLP (1.7 million parameters) - 0.8431 / 0.8303 / 0.8233 (17 epochs)
- 300D LSTM + Translate + 3 MLP (1.9 million parameters) - 0.8551 / 0.8286 / 0.8229 (23 epochs)
Following Liu et al. 2016, I don't update the GloVe embeddings during training.
Unlike Liu et al. 2016, I don't initialize out of vocabulary embeddings randomly and instead leave them zeroed.
The jokingly named SumRNN (summation of word embeddings) is 10-11x faster than the GRU or LSTM.
Original numbers for sum / LSTM from Bowman et al. '15 and Bowman et al. '16
=-=-=
100D Sum + GloVe - 0.793 / 0.753
100D LSTM + GloVe - 0.848 / 0.776
300D LSTM + GloVe - 0.839 / 0.806
'''

import keras
import keras.backend as K
from keras.callbacks import EarlyStopping, ModelCheckpoint
from keras.layers import merge, recurrent, Dense, Input, Dropout, TimeDistributed
from keras.layers.embeddings import Embedding
from keras.layers.normalization import BatchNormalization
from keras.layers.wrappers import Bidirectional
from keras.models import Model
from keras.preprocessing.sequence import pad_sequences
from keras.preprocessing.text import Tokenizer
from keras.regularizers import l2
from keras.utils import np_utils

Using TensorFlow backend.


In [2]:
def extract_tokens_from_binary_parse(parse):
    return parse.replace('(', ' ').replace(')', ' ').replace('-LRB-', '(').replace('-RRB-', ')').split()

def yield_examples(fn, skip_no_majority=True, limit=None):
      for i, line in enumerate(codecs.open(fn, encoding='utf-8')):#open(fn)):
        if limit and i > limit: break
        data = json.loads(line)
        label = data['gold_label']
        s1 = ' '.join(extract_tokens_from_binary_parse(data['sentence1_binary_parse']))
        s2 = ' '.join(extract_tokens_from_binary_parse(data['sentence2_binary_parse']))
        if skip_no_majority and label == '-': continue
        yield (label, s1, s2)

def get_data(fn, limit=None):
    fn = os.path.join('snli/snli_1.0',fn)
    raw_data = list(yield_examples(fn=fn, limit=limit))
    left = [s1 for _, s1, s2 in raw_data]
    right = [s2 for _, s1, s2 in raw_data]
    print(max(len(x.split()) for x in left))
    print(max(len(x.split()) for x in right))

    LABELS = {'contradiction': 0, 'neutral': 1, 'entailment': 2}
    Y = np.array([LABELS[l] for l, s1, s2 in raw_data])
    Y = np_utils.to_categorical(Y, len(LABELS))

    return left, right, Y


In [4]:
#print(len(training[1]))

In [45]:
training = get_data('snli_1.0_train.jsonl')
validation = get_data('snli_1.0_dev.jsonl')
test = get_data('snli_1.0_test.jsonl')

82
62
59
55
57
30


In [63]:
tokenizer = Tokenizer(lower=False, filters='')

def fix_Keras_issue1072():
    # issue: https://github.com/fchollet/keras/issues/1072
    
    from string import maketrans
    def text_to_word_sequence2(text, filters='!"#$%&()*+,-./:;<=>?@[\\]^_`{|}~\t\n', lower=True, split=" "):
        if lower: text = text.lower()
        # <<<
        # text = text.translate(maketrans(filters, split * len(filters)))
        # >>>
        # try:
        #    text = unicode(text, "utf-8")
        # except TypeError:
        #    pass
        text = text.translate({ord(c): ord(t) for c,t in  zip(filters, split*len(filters))})
        # ===
        
        seq = text.split(split)
        return [i for i in seq if i]
    
    keras.preprocessing.text.text_to_word_sequence = text_to_word_sequence2

fix_Keras_issue1072()

tokenizer.fit_on_texts(itertools.chain(training[0],training[1]))

In [65]:
def test1():
    from string import maketrans
    filters='!"#$%&()*+,-./:;<=>?@[\\]^_`{|}~\t\n'
    return {ord(c): ord(t) for c,t in  zip(filters, ' '*len(filters)) }#maketrans(filters, ' ' * len(filters))
    
#test1()
    

In [66]:
# Lowest index from the tokenizer is 1 - we need to include 0 in our vocab count
VOCAB = len(tokenizer.word_counts) + 1
LABELS = {'contradiction': 0, 'neutral': 1, 'entailment': 2}
#RNN = recurrent.LSTM
#RNN = lambda *args, **kwargs: Bidirectional(recurrent.LSTM(*args, **kwargs))
#RNN = recurrent.GRU
#RNN = lambda *args, **kwargs: Bidirectional(recurrent.GRU(*args, **kwargs))
# Summation of word embeddings
RNN = recurrent.LSTM
LAYERS = 1
USE_GLOVE = True
TRAIN_EMBED = False
EMBED_HIDDEN_SIZE = 300
SENT_HIDDEN_SIZE = 300
BATCH_SIZE = 512
PATIENCE = 4 # 8
MAX_EPOCHS = 42
MAX_LEN = 42
DP = 0.2
L2 = 4e-6
ACTIVATION = 'relu'
OPTIMIZER = 'rmsprop'
print('RNN / Embed / Sent = {}, {}, {}'.format(RNN, EMBED_HIDDEN_SIZE, SENT_HIDDEN_SIZE))
print('GloVe / Trainable Word Embeddings = {}, {}'.format(USE_GLOVE, TRAIN_EMBED))

RNN / Embed / Sent = <class 'keras.layers.recurrent.LSTM'>, 300, 300
GloVe / Trainable Word Embeddings = True, False


## Model

In [67]:
to_seq = lambda X: pad_sequences(tokenizer.texts_to_sequences(X), maxlen=MAX_LEN)
prepare_data = lambda data: (to_seq(data[0]), to_seq(data[1]), data[2])

training = prepare_data(training)
validation = prepare_data(validation)
test = prepare_data(test)

In [70]:
print('Build model...')
print('Vocab size =', VOCAB)

GLOVE_STORE = 'precomputed_glove.weights'
if USE_GLOVE:
    if not os.path.exists(GLOVE_STORE + '.npy'):
        print('Computing GloVe')

        embeddings_index = {}
        with open(os.path.join('glove.6B', 'glove.840B.300d.txt')) as f:
            for line in f:
                values = line.split(' ')
                word = values[0]
                coefs = np.asarray(values[1:], dtype='float32')
                embeddings_index[word] = coefs

        # prepare embedding matrix
        embedding_matrix = np.zeros((VOCAB, EMBED_HIDDEN_SIZE))
        for word, i in tokenizer.word_index.items():
            embedding_vector = embeddings_index.get(word)
            if embedding_vector is not None:
                # words not found in embedding index will be all-zeros.
                embedding_matrix[i] = embedding_vector
            else:
                print('Missing from GloVe: {}'.format(word))
  
        np.save(GLOVE_STORE, embedding_matrix)

    print('Loading GloVe')
    embedding_matrix = np.load(GLOVE_STORE + '.npy')

    print('Total number of null word embeddings:')
    print(np.sum(np.sum(embedding_matrix, axis=1) == 0))

    embed = Embedding(VOCAB, EMBED_HIDDEN_SIZE, weights=[embedding_matrix], input_length=MAX_LEN, trainable=TRAIN_EMBED)
else:
    embed = Embedding(VOCAB, EMBED_HIDDEN_SIZE, input_length=MAX_LEN)

rnn_kwargs = dict(output_dim=SENT_HIDDEN_SIZE, dropout_W=DP, dropout_U=DP)
SumEmbeddings = keras.layers.core.Lambda(lambda x: K.sum(x, axis=1), output_shape=(SENT_HIDDEN_SIZE, ))

translate = TimeDistributed(Dense(SENT_HIDDEN_SIZE, activation=ACTIVATION))

premise = Input(shape=(MAX_LEN,), dtype='int32')
hypothesis = Input(shape=(MAX_LEN,), dtype='int32')

prem = embed(premise)
hypo = embed(hypothesis)

prem = translate(prem)
hypo = translate(hypo)

if RNN and LAYERS > 1:
    for l in range(LAYERS - 1):
        rnn = RNN(return_sequences=True, **rnn_kwargs)
        prem = BatchNormalization()(rnn(prem))
        hypo = BatchNormalization()(rnn(hypo))
rnn = SumEmbeddings if not RNN else RNN(return_sequences=False, **rnn_kwargs)
prem = rnn(prem)
hypo = rnn(hypo)
prem = BatchNormalization()(prem)
hypo = BatchNormalization()(hypo)

joint = merge([prem, hypo], mode='concat')
joint = Dropout(DP)(joint)
for i in range(3):
    joint = Dense(2 * SENT_HIDDEN_SIZE, activation=ACTIVATION, W_regularizer=l2(L2) if L2 else None)(joint)
    joint = Dropout(DP)(joint)
    joint = BatchNormalization()(joint)

pred = Dense(len(LABELS), activation='softmax')(joint)

model = Model(input=[premise, hypothesis], output=pred)
model.compile(optimizer=OPTIMIZER, loss='categorical_crossentropy', metrics=['accuracy'])

model.summary()

Build model...
Vocab size = 42391
Computing GloVe
Missing from GloVe: nicing
Missing from GloVe: Appearal
Missing from GloVe: Lothbrok
Missing from GloVe: kniiting
Missing from GloVe: equpitment
Missing from GloVe: sailboarder
Missing from GloVe: robo-Raid
Missing from GloVe: bolloons
Missing from GloVe: seidwalk
Missing from GloVe: climbes
Missing from GloVe: koninck
Missing from GloVe: dacshund
Missing from GloVe: photoboms
Missing from GloVe: basedball
Missing from GloVe: drum-up
Missing from GloVe: dsits
Missing from GloVe: barrel-flower
Missing from GloVe: drssing
Missing from GloVe: humbee
Missing from GloVe: selfi
Missing from GloVe: elablorate
Missing from GloVe: Phinatic
Missing from GloVe: eperiments
Missing from GloVe: bridsmaid
Missing from GloVe: water-wings
Missing from GloVe: hsoe
Missing from GloVe: seideways
Missing from GloVe: cosutme
Missing from GloVe: sitcks
Missing from GloVe: instrutment
Missing from GloVe: holding-up
Missing from GloVe: furocious
Missing from Gl

In [ ]:
print('Training')
_, tmpfn = tempfile.mkstemp()
# Save the best model during validation and bail out of training early if we're not improving
callbacks = [EarlyStopping(patience=PATIENCE), ModelCheckpoint(tmpfn, save_best_only=True, save_weights_only=True)]
model.fit([training[0], training[1]], training[2], batch_size=BATCH_SIZE, nb_epoch=MAX_EPOCHS, validation_data=([validation[0], validation[1]], validation[2]), callbacks=callbacks)

# Restore the best found model during validation
model.load_weights(tmpfn)

loss, acc = model.evaluate([test[0], test[1]], test[2], batch_size=BATCH_SIZE)
print('Test loss / test accuracy = {:.4f} / {:.4f}'.format(loss, acc))

Training
Train on 549367 samples, validate on 9842 samples
Epoch 1/42
549367/549367 [==============================] - 4520s - loss: 0.8213 - acc: 0.6393 - val_loss: 0.6677 - val_acc: 0.7279

In [76]:
loss, acc = model.evaluate([test[0], test[1]], test[2], batch_size=BATCH_SIZE)
print('Test loss / test accuracy = {:.4f} / {:.4f}'.format(loss, acc))

9824/9824 [==============================] - 48s     

In [74]:
model.save_weights("model.h5")

In [113]:
def agree_or_contradict(stmt1, stmt2):
    seqs = tokenizer.texts_to_sequences([stmt1,stmt2])
    print(seqs)
    data = pad_sequences(seqs, maxlen=MAX_LEN)
    return model.predict([data[0].reshape(1,MAX_LEN), data[1].reshape(1,MAX_LEN)])

def decode_label(label):
    LABELS = {'contradiction': 0, 'neutral': 1, 'entailment': 2}
    raw_data = [('contradiction',None,None), ('neutral',None,None), ('entailment',None,None)]
    Y = np.array([LABELS[l] for l, s1, s2 in raw_data])
#     print(Y)
    Y = np_utils.to_categorical(Y, len(LABELS))
    print(Y)
decode_label(None)    
    
print(agree_or_contradict(u'A brown fox jump off the cliff.', u'the brown fox is flying.'))
print(agree_or_contradict(u'wall is cheap', u'wall is expensive'))
print(agree_or_contradict(
    u'There are three men in this picture, two are on motorbikes, \
one of the men has a large piece of furniture on the back of his bike, \
the other is about to be handed a piece of paper by a man in a white shirt.',
    u'A man is receiving a piece of paper.'))
print(agree_or_contradict(u'a man walking the dog', u'a woman reading a book'))

[[ 1.  0.  0.]
 [ 0.  1.  0.]
 [ 0.  0.  1.]]
[[3, 78, 6655, 320, 124, 6], [6, 78, 6655, 5]]
[[ 0.32584363  0.24012537  0.43403095]]
[[130, 5, 7839], [130, 5, 2904]]
[[ 0.01245843  0.4748314   0.51271009]]
[[53, 10, 184, 29, 4, 435, 50, 10, 8, 56, 11, 6, 29, 68, 2, 66, 349, 11, 1558, 8, 6, 182, 11, 21, 6, 64, 5, 169, 16, 208, 2581, 2, 349, 11, 389, 49, 2, 7, 4, 2, 27], [3, 7, 5, 2771, 2, 349, 11]]
[[ 0.02142148  0.08909965  0.8894788 ]]
[[2, 7, 41, 6, 33], [2, 14, 202, 2, 294]]
[[  9.99994397e-01   5.12713586e-06   5.30388888e-07]]


In [117]:
import pickle
with open('tokenizer.pickle', 'w') as f_tok:
    pickle.dump(tokenizer, f_tok)

In [119]:
tokenizer.texts_to_sequences([u'that is nice'])

[[99, 5, 911]]

In [120]:
model.summary()

____________________________________________________________________________________________________
Layer (type)                     Output Shape          Param #     Connected to                     
input_1 (InputLayer)             (None, 42)            0                                            
____________________________________________________________________________________________________
input_2 (InputLayer)             (None, 42)            0                                            
____________________________________________________________________________________________________
embedding_1 (Embedding)          (None, 42, 300)       12717300    input_1[0][0]                    
                                                                   input_2[0][0]                    
____________________________________________________________________________________________________
timedistributed_1 (TimeDistribut (None, 42, 300)       90300       embedding_1[0][0]       

In [123]:
with open('test_dataset.pickle', 'w') as f:
    pickle.dump(test, f)